In [1]:
from models import unetnc
import numpy as np
import pickle as pkl
import cv2
import torch
import cv2

DATA_PATH = 'Dataset Preview/Inv3D preview/data/'
# warped WC sind die 3D Koordinaten nach dem ersten Modell bzw. dewarpnet -> Ground Truth

In [2]:

trainimage_gt = np.load(DATA_PATH + 'train/06/warped_WC.npz')

In [8]:
img_t.shape

(256, 256, 3)

In [3]:
img_t = trainimage_gt['warped_WC']
#img_t = torch.from_numpy(img).float()

In [5]:
#normalize label
lbl = img_t
msk=((lbl[:,:,0]!=0)&(lbl[:,:,1]!=0)&(lbl[:,:,2]!=0)).astype(np.uint8)*255
xmx, xmn, ymx, ymn,zmx, zmn= 1.2539363, -1.2442188, 1.2396319, -1.2289206, 0.6436657, -0.67492497   # calculate from all the wcs
lbl[:,:,0]= (lbl[:,:,0]-zmn)/(zmx-zmn)
lbl[:,:,1]= (lbl[:,:,1]-ymn)/(ymx-ymn)
lbl[:,:,2]= (lbl[:,:,2]-xmn)/(xmx-xmn)
lbl=cv2.bitwise_and(lbl,lbl,mask=msk)
#lbl = cv2.resize(lbl, self.img_size, interpolation=cv2.INTER_NEAREST)
lbl = lbl.transpose(2, 0, 1)   # NHWC -> NCHW
lbl = np.array(lbl, dtype=np.float64)
lbl

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [7]:
lbl.shape

(3, 256, 256)

In [79]:
type(trainimage_gt['warped_WC'])

numpy.ndarray

In [82]:
np.load(DATA_PATH + 'train/06/warped_document.png')

ValueError: Cannot load file containing pickled data when allow_pickle=False

In [87]:
from numpy import asarray
from PIL import Image
image = Image.open(DATA_PATH + 'train/01/warped_document.png')
# convert image to numpy array
img = asarray(image)
#img = m.imresize(img, self.img_size) # uint8 with RGB mode
if img.shape[-1] == 4:
    img=img[:,:,:3]   # Discard the alpha channel  
img = img[:, :, ::-1] # RGB -> BGR
# plt.imshow(img)
# plt.show()
img = img.astype(float) / 255.0
img = img.transpose(2, 0, 1) # NHWC -> NCHW


In [89]:
img.shape

(3, 256, 256)

In [6]:
DATA_PATH = 'Dataset Preview/Inv3D preview/data/'

from custom_dataset import CustomImageDataset_wc
train_dataset = CustomImageDataset_wc(data_dir=DATA_PATH+'train/', transform=True)
#train_dataset = CustomImageDataset_wc(data_dir=DATA_PATH+'train/', transform=False)

from models import unetnc
model = unetnc.UnetGenerator(input_nc=3, output_nc=6, num_downs=3)
#model.to('cuda')
from torch.autograd import Variable

In [7]:
#for param in model.parameters():
 #   param.requires_grad = False
#for param in model.parameters():
    #print(param)

In [8]:
from torch.utils import data
#data_loader = get_loader('doc3dwc')
#data_path = args.data_path
#t_loader = data_loader(data_path, is_transform=True, img_size=(args.img_rows, args.img_cols), augmentations=True)
#v_loader = data_loader(data_path, is_transform=True, split='val', img_size=(args.img_rows, args.img_cols))

#n_classes = t_loader.n_classes
trainloader = data.DataLoader(train_dataset, batch_size=1, num_workers=8, shuffle=True)
#valloader = data.DataLoader(v_loader, batch_size=args.batch_size, num_workers=8)

In [9]:
for image, labels in trainloader:
    #print(item)
    pass
#img = Variable(item['image'])#.to('cuda'))  #images = Variable(images.cuda())
#img.size()

In [12]:
labels

{'wc_gt': tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
 
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
 
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]]]),
 'curvature_gt': ['TODO'],
 'text_mask_gt': ['TODO'],
 'local_angle_gt': ['TODO']}

In [10]:
output = model(image)

In [15]:
output.shape

torch.Size([1, 6, 256, 256])

In [17]:
output[0].shape

torch.Size([2, 256, 256])

In [ ]:
from torch.autograd import Variable
for i, (images, labels) in enumerate(trainloader):
    print(i)
    print(images)
    print(labels)
    #images = Variable(images.cuda())
    #labels = Variable(labels.cuda())

    #optimizer.zero_grad()
    outputs = model(images)
    '''pred=htan(outputs)
    g_loss=gloss(pred, labels)
    l1loss = loss_fn(pred, labels)
    loss=l1loss#+(0.2*g_loss)
    avg_l1loss+=float(l1loss)
    avg_gloss+=float(g_loss)
    avg_loss+=float(loss)
    train_mse+=float(MSE(pred, labels).item())

    loss.backward()
    optimizer.step()
    global_step+=1'''
    print('lakshooOOOO')
    print(outputs)

### Pytorch Lightning Training

In [11]:
import pytorch_lightning as pl
DATA_PATH = 'Dataset Preview/Inv3D preview/data/'

from custom_dataset import CustomImageDataset_wc
dataset = CustomImageDataset_wc(data_dir=DATA_PATH+'train/', transform=True)
#train_dataset = CustomImageDataset_wc(data_dir=DATA_PATH+'train/', transform=False)
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset, batch_size= 4)


from models import unetnc
model = unetnc.UnetGenerator(input_nc=3, output_nc=7, num_downs=3)



# most basic trainer, uses good defaults (auto-tensorboard, checkpoints, logs, and more)
# trainer = pl.Trainer(gpus=8) (if you have GPUs)
trainer = pl.Trainer(gpus=1)
#trainer.fit(model, train_loader)

# Wie sieht der Batch aus, der vom trainer und train_loader generiert wird?

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [12]:
for batch in train_loader:
    images, labels = batch

output = model(images)

In [13]:
import torch.nn as nn
#import grad_loss
# Losses
#MSE = nn.MSELoss()
loss_fn = nn.L1Loss()
#gloss= grad_loss.Gradloss(window_size=5,padding=2)

# Activation
htan = nn.Hardtanh(0,1.0)

In [21]:
def split_tensors(model_output):
    batch_size = model_output.shape[0]
    wc_coordinates_pred = torch.empty(batch_size,3,256,256)
    #angle_pred = torch.empty(batch_size,4,256,256)
    phi_xx = torch.empty(batch_size,1,256,256)
    phi_xy = torch.empty(batch_size,1,256,256)
    phi_yx = torch.empty(batch_size,1,256,256)
    phi_yy = torch.empty(batch_size,1,256,256)

    idx = 0
    for output in model_output:
        
        wc_coordinates_pred[idx] = output[:3]
        #angle_pred[idx] = output[3:]
        phi_xx[idx] = output[3]
        phi_xy[idx] = output[4]
        phi_yx[idx] = output[5]
        phi_yy[idx] = output[6]

        idx += 1

    angle_pred = {'phi_xx' : phi_xx, 'phi_xy' : phi_xy, 'phi_yx' : phi_yx, 'phi_yy' : phi_yy,}
    return wc_coordinates_pred, angle_pred

In [22]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.L1Loss()
wc_coordinates_output, angle_output = split_tensors(output)

#wc_coordinates_pred=htan(wc_coordinates_output)
#l1_loss = loss_fn(wc_coordinates_pred, labels['wc_gt'])

In [25]:
angle_output['theta_xx'].shape

torch.Size([3, 1, 256, 256])

In [137]:
l1_loss 

tensor(0.2801, grad_fn=<L1LossBackward>)

In [138]:

loss = l1_loss #+ torch.tensor(0.25, requires_grad=True)

In [139]:
loss.backward()
optimizer.step()
#optimizer.zero_grad()

In [141]:
l1_loss.grad

<ipython-input-141-291d24f22676>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  l1_loss.grad


In [26]:
theta_x = torch.atan2(angle_output['phi_xx'], angle_output['phi_xy'])

tensor([[[[-2.3387,  1.8802, -3.0584,  ...,  2.3485, -3.0838,  1.8430],
          [ 1.0988,  2.4272,  0.0600,  ...,  1.4874, -0.4773, -0.5839],
          [ 1.1703,  2.1403,  0.9957,  ...,  1.9287,  0.8192, -2.4067],
          ...,
          [-0.9434,  3.1323, -0.2417,  ...,  2.9774,  0.6064, -2.6378],
          [ 0.0099, -2.8343, -1.4282,  ...,  2.2990,  1.9182,  2.6005],
          [ 1.0062, -3.1223,  0.5710,  ...,  2.4470,  0.3611, -0.6768]]],


        [[[-2.5209,  1.1893, -2.5897,  ...,  2.3591,  2.8945,  1.1009],
          [ 1.3222, -3.0580, -0.1832,  ...,  1.9137, -0.6337, -2.9021],
          [ 2.5742,  2.0284,  1.1411,  ...,  2.2434,  0.8028, -2.9189],
          ...,
          [-0.7203,  2.6766, -0.1862,  ...,  2.5575,  0.6216, -2.8381],
          [ 0.3242,  2.4139,  1.7511,  ...,  1.7789,  1.3775,  2.7694],
          [ 0.9446,  2.9962,  0.5149,  ..., -2.4320,  0.1336, -1.1009]]],


        [[[-2.3946,  1.2582, -2.7747,  ...,  2.3261,  2.8953,  1.0995],
          [ 1.1989,  2.198

In [97]:
model.parameters().grad

AttributeError: 'generator' object has no attribute 'grad'

In [144]:
torch.sum(l1_loss,torch.tensor(0.25, requires_grad=True))

TypeError: sum() received an invalid combination of arguments - got (Tensor, Tensor), but expected one of:
 * (Tensor input, *, torch.dtype dtype)
 * (Tensor input, tuple of names dim, bool keepdim, *, torch.dtype dtype, Tensor out)
 * (Tensor input, tuple of ints dim, bool keepdim, *, torch.dtype dtype, Tensor out)


In [215]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.linear = nn.Linear(2,2)
        nn.

    def forward(self, x):
        x = F.relu(self.linear(x))
        return x


net = Net()

In [148]:
print(net)

Net(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)


tensor([[1.4341, 0.0000]], grad_fn=<ReluBackward0>)


In [304]:
input = torch.randn(1, 2, requires_grad=True)
out = net(input)
print(input)
print(out)

tensor([[0.8760, 1.4145]], requires_grad=True)
tensor([[0.1553, 0.0000]], grad_fn=<ReluBackward0>)


In [305]:
loss_fn = nn.L1Loss()

emp = torch.empty(1,1)
emp[0][0] = 2
#emp[0][1] = 2'''

emp2 = torch.empty(1,2)
emp2[0][0] = 1
emp2[0][1] = 2
# Activation
htan = nn.Hardtanh(0,1.0)
t1 = torch.empty(1,1)
t1[0] = out[0][0]
t2 = torch.empty(1,1)
t2[0] = out[0][1]
#pred=htan(out)
l1_loss = loss_fn((t1+2) * 5 , emp)

In [306]:
l1_loss.backward()

In [299]:
l1_loss.grad

<ipython-input-299-291d24f22676>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  l1_loss.grad


In [307]:
input.grad

tensor([[0.2555, 2.7904]])